# Peer Review Assignment - Data Engineer - ETL

## Objectives
In this final part you will:

* Run the ETL process
* Extract bank and market cap data from the JSON file bank_market_cap.json
* Transform the market cap currency using the exchange rate data
* Load the transformed data into a seperate CSV

### Imports

In [149]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section



In [150]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-05-15 15:26:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.3’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-05-15 15:26:05 (533 MB/s) - ‘bank_market_cap_1.json.3’ saved [2815/2815]

--2023-05-15 15:26:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20

In [151]:
# check the downloaded files list
import os
os.listdir("/kaggle/working/")

['bank_market_cap_2.json.1',
 'bank_market_cap_gbp.csv',
 'bank_data.csv',
 '.virtual_documents',
 'bank_market_cap_1.json.2',
 'exchange_rates.csv.3',
 'bank_market_cap_2.json.3',
 'bank_market_cap_1.json.1',
 'logfile.txt',
 'bank_market_cap_1.json.3',
 'exchange_rates.csv',
 'bank_market_cap_2.json',
 'bank_market_cap_1.json',
 'exchange_rates.csv.2',
 'bank_market_cap_2.json.2',
 'exchange_rates.csv.1',
 '__notebook_source__.ipynb']

## Extract
### JSON Extract Function
This function will extract JSON files.



In [152]:
# create extract function
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

### Extract Function
Define the extract function that finds JSON file bank_market_cap_1.json and calls the function created above to extract data from them. Store the data in a pandas dataframe. Use the following list for the columns.

In [153]:
# set columns for a dataframe
columns=['Name','Market Cap (US$ Billion)']

In [154]:
# set the dataframe
bank_data=pd.DataFrame(columns)

# mention the file
extract_file = 'bank_market_cap_1.json'

# set the extract function
def extract(extract_file):
    bank_data = extract_from_json(extract_file)
    return bank_data

**Question 1**. Load the file **exchange_rates.csv** as a dataframe and find the exchange rate for British pounds with the symbol GBP, store it in the variable **exchange_rate**, you will be asked for the number. Hint: set the parameter index_col to 0.

In [155]:
# Write your code here
df = pd.read_csv("exchange_rates.csv")
print(df.head(10))

exchange_rate = df.loc[9,"Rates"] # loc to select row 9, column 1 = GBP
print('Exchange rate for British pounds - GBP = ')
print(exchange_rate)

  Unnamed: 0      Rates
0        AUD   1.297088
1        BGN   1.608653
2        BRL   5.409196
3        CAD   1.271426
4        CHF   0.886083
5        CNY   6.483139
6        CZK  21.510117
7        DKK   6.119757
8        EUR   0.822504
9        GBP   0.732398
Exchange rate for British pounds - GBP = 
0.7323984208000001



### Transform

Using exchange_rate and the exchange_rates.csv file find the exchange rate of USD to GBP. 

Write a transform function that

1.     Changes the Market Cap (US$ Billion) column from USD to GBP

1.     Rounds the Market Cap (US$ Billion) column to 3 decimal places

1.     Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)


In [156]:
def transform(data):
    # Write your code here
    # print( data['Market Cap (US$ Billion)'])
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange_rate, 3)
    # print( data['Market Cap (US$ Billion)'])
    data.rename(columns = {'Market Cap (US$ Billion)':'Market Cap (GBP$ Billion)'}, inplace = True)
    # print( data['Market Cap (US$ Billion)'])
    return data

### Load¶
Create a function that takes a dataframe and load it to a csv named **bank_market_cap_gbp.csv**. Make sure to set index to False.

In [157]:
# load the bank_data dataframe to .csv file
def load(dataframe_name):
    dataframe_name.to_csv('bank_market_cap_gbp.csv',index=False)

In [158]:
# test loader
bank_data.to_csv("bank_data.csv", index = False)


### Logging Function

Write the logging function log to log your data:


In [159]:
# create log function
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

### Running the ETL Process
Log the process accordingly using the following **"ETL Job Started"** and **"Extract phase Started"**

In [160]:
# set the logging commands
log('ETL Job Started')
log('Extract Phase Started')

### Extract
**Question 2** Use the function extract, and print the first 5 rows, take a screen shot:

In [161]:
# Call the extract function here
extract_data = extract('bank_market_cap_1.json')
# Print the extracted rows here
extract_data.head(5)

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399



Log the data as **"Extract phase Ended"**


In [162]:
# set log for extract phase
log('Extract phase Ended')


### Transform

Log the following "Transform phase Started"

In [163]:
# set log for transform phase
log('Transform phase Started')

**Question 3** Use the function transform and print the first 5 rows of the output, take a screen shot:

In [164]:
# Call the transform function
transform_data = transform(extract_data)
# Print the first 5 rows of data
transform_data.head(5)

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log your data "Transform phase Ended"

In [165]:
# set log for transform phase end
log('Transform phase Ended')

### Load
Log the following "Load phase Started".

In [166]:
# set log for load phase
log('Load Phase Started')

Call the load function

In [167]:
# load transformed data
load(transform_data)

Log the following "Load phase Ended".

In [168]:
# set the extract function
def extract(extract_file):
    data = pd.read_csv(extract_file)
    return data

# Call the extract function
currency = extract('exchange_rates.csv')
# Print the extracted rows
currency.head(5)

,Unnamed: 0,Rates
0,AUD,1.297088
1,BGN,1.608653
2,BRL,5.409196
3,CAD,1.271426
4,CHF,0.886083


In [169]:
# set log for load phase end
log('Load phase Ended')